In [1]:
import os 
import sys

In [4]:
# copy one rendered image of all models to a folder
model_dir = "/media/junting/SSD_data/mani_skill2_data"
src_img = "parts_render_after_merging/0.png"
target_dir = "/media/junting/SSD_data/mani_skill2_data/model_thumbnails"
if not os.path.exists(target_dir):
    os.mkdir(target_dir)

# each model is a random 4-digit number, check format and list dir and sort
model_list = [ item for item in os.listdir(model_dir) if item.isdigit()]
for model in model_list:
    src = os.path.join(model_dir, model, src_img)
    dst = os.path.join(target_dir, model+".png")
    os.system("cp {} {}".format(src, dst))


cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4000/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4001/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4003/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4006/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4008/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4009/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4010/parts_render_after_merging/0.png': No such file or directory
cp: cannot stat '/media/junting/SSD_data/mani_skill2_data/4011/parts_render_after_merging/0.png': No such file or directory
cp: cann

In [14]:
import numpy as np 
from scipy.spatial.transform import Rotation as R

In [50]:
import numpy as np

import numpy as np

def flip_pose_matrix_with_point(original_pose_matrix, plane_normal, plane_point):
    """
    Compute the flipped pose matrix with respect to a plane defined by a normal vector and a point on the plane.

    Args:
        original_pose_matrix (numpy.ndarray): The original pose matrix to be flipped.
        plane_normal (numpy.ndarray): The normal vector of the plane.
        plane_point (numpy.ndarray): A point on the plane.

    Returns:
        numpy.ndarray: The flipped pose matrix.
    """
    # Ensure the plane normal is a unit vector
    plane_normal = plane_normal / np.linalg.norm(plane_normal)

    # Extract the rotation matrix and translation vector from the pose matrix
    orientation_matrix = original_pose_matrix[:3, :3]
    translation_vector = original_pose_matrix[:3, 3]

    # Calculate the reflected position
    reflected_position = translation_vector - 2 * np.dot(translation_vector - plane_point, plane_normal) * plane_normal

    # Calculate the orthogonal projection of the rotation matrix onto the plane
    projection = orientation_matrix - np.outer(plane_normal, plane_normal) @ orientation_matrix

    # Calculate the reflected rotation matrix
    reflected_orientation = -projection

    flipped_pose_matrix = np.eye(4)
    flipped_pose_matrix[:3, :3] = reflected_orientation
    flipped_pose_matrix[:3, 3] = reflected_position

    return flipped_pose_matrix



In [51]:
"""
Original pose: 
  position: 
    x: 0.5849478244781494
    y: -1.4837896823883057
    z: 1.7380565404891968
  orientation: 
    x: -0.2087007462978363
    y: 0.09191908687353134
    z: 0.8641879558563232
    w: 0.4485242962837219

"""
# original pose
position = np.array([0.5849478244781494, -1.4837896823883057, 1.7380565404891968])
quaternion = np.array([-0.2087007462978363, 0.09191908687353134, 0.8641879558563232, 0.4485242962837219])
pose_matrix = np.eye(4)
pose_matrix[:3, 3] = position
# pose_matrix[:3, :3] = R.from_quat(quaternion).as_matrix()
pose_matrix[:3, :3] = R.from_euler('xyz', [0.0,  0.45913096,  2.17671885], degrees=False).as_matrix()

# mirror plane
mirror_normal = np.array([0, 1, 0])
mirror_point = np.array([0, 0, 0])

mirrored_pose_matrix = flip_pose_matrix_with_point(pose_matrix, mirror_normal, mirror_point)


In [52]:
# convert original and mirrored quaternion to euler in radians
original_euler = R.from_matrix(pose_matrix[:3, :3]).as_euler('xyz', degrees=False)
mirrored_euler = R.from_matrix(mirrored_pose_matrix[:3, :3]).as_euler('xyz', degrees=False)
print("originnal position",  pose_matrix[:3, 3])
print("original euler: ", original_euler)
print("mirrored position", mirrored_pose_matrix[:3, 3])
print("mirrored euler: ", mirrored_euler)


originnal position [ 0.58494782 -1.48378968  1.73805654]
original euler:  [0.         0.45913096 2.17671885]
mirrored position [0.58494782 1.48378968 1.73805654]
mirrored euler:  [ 2.95282279 -0.50245442  0.70674306]
